# K Fold validation

The follow codes aim to evaluate models based on FE data version 2 where the use of FE were minimised. The models will be evaluated using 5 fold splitting and its RMSE mean and variance are compared.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import time
import torch


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,  mean_absolute_error
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_score, average_precision_score, log_loss

In [7]:
# Train data
data = pd.read_csv('C:/Users/susum/Documents/year5_sem1/QBUS3820/Group_Assignment/final_version/data_train.csv')
pd.set_option('display.max_columns', len(data.columns))
data.head()

,host_since,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,zipcode,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,security_deposit_perc,cleaning_fee_perc,guests_included,extra_people_perc,minimum_nights,maximum_nights,availability_90,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,reviews_per_month,price,flexible,moderate,Ashfield,Auburn,Bankstown,Blacktown,Botany_Bay,Burwood,Camden,Campbelltown,Canada_Bay,Canterbury,City_Of_Kogarah,Fairfield,Holroyd,Hornsby,Hunters_Hill,Hurstville,Ku_Ring_Gai,Lane_Cove,Leichhardt,Liverpool,Manly,Marrickville,Mosman,North_Sydney,Parramatta,Penrith,Pittwater,Randwick,Rockdale,Ryde,Strathfield,Sutherland_Shire,Sydney,The_Hills_Shire,Warringah,Waverley,Willoughby,Woollahra,Aparthotel,Apartment,Bed_and_breakfast,Boat,Boutique_hotel,Bungalow,Cabin,Camper_RV,Condominium,Cottage,Earth_house,Farm_stay,Guest_suite,Guesthouse,Hostel,Hotel,House,Island,Loft,Other,Serviced_apartment,Tiny_house,Townhouse,Villa,Yurt,Entire_home_apt,Hotel_room,Private_room,Shared_room,Couch,Futon,Pull_out_Sofa,Real_Bed,Missing,a_few_days_or_more,within_a_day,within_a_few_hours,within_an_hour,sent_id_dummy,sesame_dummy,sesame_offline_dummy,weibo_dummy,kba_dummy,zhima_selfie_dummy,manual_online_dummy,manual_offline_dummy,google_dummy,work_email_dummy,facebook_dummy,identity_manual_dummy,selfie_dummy,offline_government_id_dummy,jumio_dummy,reviews_dummy,government_id_dummy,email_dummy,phone_dummy,Washer_Dryer_dummy,Breakfast_bar_dummy,Infinity_pool_dummy,Parking_dummy,Projector_and_screen_dummy,Bidet_dummy,Shared_gym_dummy,Touchless_faucets_dummy,Pool_cover_dummy,Garage_parking_dummy,Private_gym_dummy,Bathtub_with_bath_chair_dummy,Brick_oven_dummy,Gated_property_dummy,Propane_barbeque_dummy,Apple_TV_dummy,Security_system_dummy,Warming_drawer_dummy,Stand_alone_steam_shower_dummy,Mountain_view_dummy,Ironing_Board_dummy,Ice_Machine_dummy,Gas_fireplace_dummy,Outdoor_kitchen_dummy,Jetted_tub_dummy,Steam_oven_dummy,Tennis_court_dummy,Amazon_Echo_dummy,HBO_GO_dummy,Shared_hot_tub_dummy,Ground_floor_access_dummy,Chef_s_kitchen_dummy,Dining_area_dummy,Fire_pit_dummy,Hammock_dummy,Sauna_dummy,Standing_valet_dummy,Fixed_grab_bars_for_toilet_dummy,Private_hot_tub_dummy,Heat_lamps_dummy,Private_bathroom_dummy,Murphy_bed_dummy,Exercise_equipment_dummy,Pool_toys_dummy,Shower_chair_dummy,Alfresco_bathtub_dummy,High_resolution_computer_monitor_dummy,Wine_cooler_dummy,Ski_in_Ski_out_dummy,Fixed_grab_bars_for_shower_dummy,Shared_pool_dummy,Firm_mattress_dummy,Double_oven_dummy,Private_pool_dummy,Heated_floors_dummy,Printer_dummy,Beach_view_dummy,Central_air_conditioning_dummy,Outdoor_parking_dummy,Day_bed_dummy,Gas_oven_dummy,Memory_foam_mattress_dummy,Mini_fridge_dummy,Sun_loungers_dummy,Wide_clearance_to_shower_dummy,_toilet_dummy,Heated_towel_rack_dummy,Step_free_shower_dummy,Wide_doorway_to_guest_bathroom_dummy,Soaking_tub_dummy,Formal_dining_area_dummy,Accessible_height_toilet_dummy,Table_corner_guards_dummy,Disabled_parking_spot_dummy,Wide_entrance_dummy,Other_pet_s__dummy,Wide_entryway_dummy,EV_charger_dummy,Accessible_height_bed_dummy,Kitchenette_dummy,DVD_player_dummy,Rain_shower_dummy,Terrace_dummy,Ceiling_fan_dummy,Pillow_top_mattress_dummy,Fireplace_guards_dummy,Extra_space_around_bed_dummy,En_suite_bathroom_dummy,Convection_oven_dummy,Sound_system_dummy,Breakfast_table_dummy,Bread_maker_dummy,Baby_monitor_dummy,Balcony_dummy,Doorman_dummy,Outdoor_seating_dummy,Netflix_dummy,Espresso_machine_dummy,Smart_TV_dummy,Walk_in_shower_dummy,Hot_water_kettle_dummy,Lake_access_dummy,Outlet_covers_dummy,Stair_gates_dummy,Wide_hallways_dummy,Wide_entrance_for_guests_dummy,Flat_path_to_guest_entrance_dummy,Smart_lock_dummy,Building_staff_dummy,Barbecue_utensils_dummy,Baking_sheet_dummy,Body_soap_dummy,Bath_towel_dummy,Toilet_paper_dummy,Cat_s__dummy,Bedroom_comforts_dummy,Bathroom_essent

# Split data

In [9]:
index_train, index_val  = train_test_split(data.index, train_size = 0.5, random_state=15)

# Define predictors and response
predictors = list(data.columns)
predictors.remove('price')
response = ['price']

X = data.loc[:, predictors]
y = np.log(data.loc[:, response])

# OLS

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

kf = KFold()
RMSE_train = []
RMSE_test = []
count = 0

time_start = time.time()

fold_num = kf.get_n_splits(data)

for train_index, test_index in kf.split(data):

    X_train, X_test = X.loc[train_index, predictors], X.loc[test_index, predictors]
    y_train, y_test = y.loc[train_index, response], y.loc[test_index, response]

    ols1 = LinearRegression()
    ols1.fit(X_train, y_train)

    y_pred = ols1.predict(X_test)
    y_pred_train = ols1.predict(X_train)
    
    a = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE_train.append(a)
    b = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSE_test.append(b)
    
    count += 1 
    
    print(f'\nFold number {count}')

    print(f'\nTrain MSE is {(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train RMSE is {np.sqrt(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train r2 is {r2_score(y_train, y_pred_train)}')

    print(f'\nTest MSE is {(mean_squared_error(y_test, y_pred))}')
    print(f'Test RMSE is {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'Test r2 is {r2_score(y_test, y_pred)}')
    
time_end = time.time()
    
print(f'\nThe average Train RMSE is {np.array(RMSE_train).mean()} ')
print(f'The average Test RMSE is {np.array(RMSE_test).mean()} ')
print(f'The variance Test RMSE is {np.array(RMSE_test).var()} ')
print(f'\ntime taken is {(time_end - time_start)/60} minutes')


Fold number 1

Train MSE is 0.13700559153734765
Train RMSE is 0.3701426637627006
Train r2 is 0.7426580379832018

Test MSE is 0.15138712155708678
Test RMSE is 0.38908497986569307
Test r2 is 0.726894657644515

Fold number 2

Train MSE is 0.13932992171257155
Train RMSE is 0.3732692348862568
Train r2 is 0.7438281155602922

Test MSE is 0.14368339558997628
Test RMSE is 0.3790559267311043
Test r2 is 0.717581234060622

Fold number 3

Train MSE is 0.13861621191451928
Train RMSE is 0.3723119819647486
Train r2 is 0.7412736641596354

Test MSE is 0.14390857800996976
Test RMSE is 0.3793528410463928
Test r2 is 0.7341866681466669

Fold number 4

Train MSE is 0.13936200817151415
Train RMSE is 0.37331221272751597
Train r2 is 0.742297841918385

Test MSE is 0.1408977847559682
Test RMSE is 0.3753635367959549
Test r2 is 0.7296925795506009

Fold number 5

Train MSE is 0.1379945935536218
Train RMSE is 0.37147623551665027
Train r2 is 0.7404068762738698

Test MSE is 0.14736341487745055
Test RMSE is 0.383879427

# GBR

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

kf = KFold()
RMSE_train = []
RMSE_test = []
count = 0

time_start = time.time()

fold_num = kf.get_n_splits(data)

for train_index, test_index in kf.split(data):

    X_train, X_test = X.loc[train_index, predictors], X.loc[test_index, predictors]
    y_train, y_test = y.loc[train_index, response], y.loc[test_index, response]

    gbr = GradientBoostingRegressor(learning_rate= 0.05, max_depth = 4, n_estimators= 1500, subsample = 0.8)
    gbr.fit(X_train, y_train)

    y_pred = gbr.predict(X_test)
    y_pred_train = gbr.predict(X_train)
    
    a = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE_train.append(a)
    b = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSE_test.append(b)
    
    count += 1 
    
    print(f'\nFold number {count}')

    print(f'\nTrain MSE is {(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train RMSE is {np.sqrt(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train r2 is {r2_score(y_train, y_pred_train)}')

    print(f'\nTest MSE is {(mean_squared_error(y_test, y_pred))}')
    print(f'Test RMSE is {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'Test r2 is {r2_score(y_test, y_pred)}')
    
time_end = time.time()
    
print(f'\nThe average Train RMSE is {np.array(RMSE_train).mean()} ')
print(f'The average Test RMSE is {np.array(RMSE_test).mean()} ')
print(f'The variance Test RMSE is {np.array(RMSE_test).var()} ')
print(f'\ntime taken is {(time_end - time_start)/60} minutes')

C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 1

Train MSE is 0.02997646941219951
Train RMSE is 0.1731371404759808
Train r2 is 0.9436942436705653

Test MSE is 0.10882415384639602
Test RMSE is 0.32988506156901987
Test r2 is 0.8036790878439509


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 2

Train MSE is 0.030743850479711038
Train RMSE is 0.17533924398066464
Train r2 is 0.9434743806964361

Test MSE is 0.09398730444953493
Test RMSE is 0.3065734894760715
Test r2 is 0.8152620320001818


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 3

Train MSE is 0.03126873868514709
Train RMSE is 0.17682968835901705
Train r2 is 0.9416370850521664

Test MSE is 0.1002827834814001
Test RMSE is 0.3166745703105952
Test r2 is 0.8147678118056962


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 4

Train MSE is 0.030393575197909622
Train RMSE is 0.17433753238448002
Train r2 is 0.9437975239946482

Test MSE is 0.10342981476352178
Test RMSE is 0.32160506022685925
Test r2 is 0.8015735557893325


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 5

Train MSE is 0.029925254201881746
Train RMSE is 0.1729891736551214
Train r2 is 0.9437051117981202

Test MSE is 0.1015659750902135
Test RMSE is 0.31869417172300707
Test r2 is 0.818001167934964

The average Train RMSE is 0.17452655577105278 
The average Test RMSE is 0.31868647066111055 
The variance Test RMSE is 5.693974377837494e-05 

time taken is 5.044375324249268 minutes


# LightGBM

In [14]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import lightgbm as lgb

kf = KFold()
RMSE_train = []
RMSE_test = []
count = 0

time_start = time.time()

fold_num = kf.get_n_splits(data)

for train_index, test_index in kf.split(data):

    X_train, X_test = X.loc[train_index, predictors], X.loc[test_index, predictors]
    y_train, y_test = y.loc[train_index, response], y.loc[test_index, response]

    lbst = lgb.LGBMRegressor(objective='regression', subsample = 0.8, num_leaves = 24, 
                                           n_estimators = 500, learning_rate = 0.05)

    lbst.fit(X_train, y_train)

    y_pred = lbst.predict(X_test)
    y_pred_train = lbst.predict(X_train)
    
    a = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE_train.append(a)
    b = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSE_test.append(b)
    
    count += 1 
    
    print(f'\nFold number {count}')

    print(f'\nTrain MSE is {(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train RMSE is {np.sqrt(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train r2 is {r2_score(y_train, y_pred_train)}')

    print(f'\nTest MSE is {(mean_squared_error(y_test, y_pred))}')
    print(f'Test RMSE is {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'Test r2 is {r2_score(y_test, y_pred)}')
    
    
time_end = time.time()
    
print(f'\nThe average Train RMSE is {np.array(RMSE_train).mean()} ')
print(f'The average Test RMSE is {np.array(RMSE_test).mean()} ')
print(f'The variance Test RMSE is {np.array(RMSE_test).var()} ')
print(f'\ntime taken is {(time_end - time_start)/60} minutes')



Fold number 1

Train MSE is 0.04268571849029485
Train RMSE is 0.20660522377300833
Train r2 is 0.9198220567268254

Test MSE is 0.10811764540802242
Test RMSE is 0.32881247757349846
Test r2 is 0.8049536429511124

Fold number 2

Train MSE is 0.04366323004107225
Train RMSE is 0.2089574838120718
Train r2 is 0.9197208195995369

Test MSE is 0.09783997780595195
Test RMSE is 0.3127938263552399
Test r2 is 0.8076893598036546

Fold number 3

Train MSE is 0.04411656627913672
Train RMSE is 0.21003943981818443
Train r2 is 0.9176566911935351

Test MSE is 0.10065202951732834
Test RMSE is 0.3172570401383212
Test r2 is 0.8140857779725434

Fold number 4

Train MSE is 0.04395844214067156
Train RMSE is 0.20966268657219758
Train r2 is 0.9187139626201772

Test MSE is 0.1041217434828874
Test RMSE is 0.3226790099818818
Test r2 is 0.8002461149953519

Fold number 5

Train MSE is 0.04346947136350459
Train RMSE is 0.20849333649664822
Train r2 is 0.9182259567756845

Test MSE is 0.10018439296780934
Test RMSE is 0.316

# XG Boosting

In [16]:
import xgboost as xgb

start_time = time.time()

kf = KFold()
RMSE_train = []
RMSE_test = []
count = 0

time_start = time.time()

fold_num = kf.get_n_splits(data)

for train_index, test_index in kf.split(data):

    X_train, X_test = X.loc[train_index, predictors], X.loc[test_index, predictors]
    y_train, y_test = y.loc[train_index, response], y.loc[test_index, response]

    xbst = xgb.XGBRegressor(colsample_bytree = 0.306, subsample = 1.0, n_estimators = 1975, max_depth = 6,
                            learning_rate = 0.05, reg_lambda = 1000)
    xbst.fit(X_train, y_train)

    y_pred = xbst.predict(X_test)
    y_pred_train = xbst.predict(X_train)
    
    a = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE_train.append(a)
    b = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSE_test.append(b)
    
    count += 1 
    
    print(f'\nFold number {count}')

    print(f'\nTrain MSE is {(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train RMSE is {np.sqrt(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train r2 is {r2_score(y_train, y_pred_train)}')

    print(f'\nTest MSE is {(mean_squared_error(y_test, y_pred))}')
    print(f'Test RMSE is {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'Test r2 is {r2_score(y_test, y_pred)}')
    
time_end = time.time()
    
print(f'\nThe average Train RMSE is {np.array(RMSE_train).mean()} ')
print(f'The average Test RMSE is {np.array(RMSE_test).mean()} ')
print(f'The variance Test RMSE is {np.array(RMSE_test).var()} ')
print(f'\ntime taken is {(time_end - time_start)/60} minutes')




Fold number 1

Train MSE is 0.05807454490508365
Train RMSE is 0.24098660731477103
Train r2 is 0.8909167344090998

Test MSE is 0.10498757742868647
Test RMSE is 0.32401786590971565
Test r2 is 0.8106003470980707

Fold number 2

Train MSE is 0.05898634579085063
Train RMSE is 0.24287104765873316
Train r2 is 0.8915477510378087

Test MSE is 0.09517106630899709
Test RMSE is 0.3084980815321176
Test r2 is 0.8129352734896198

Fold number 3

Train MSE is 0.057853452748761164
Train RMSE is 0.24052744697593487
Train r2 is 0.8920168742265795

Test MSE is 0.0986504658697967
Test RMSE is 0.31408671711773595
Test r2 is 0.8177828633681758

Fold number 4

Train MSE is 0.057763001016091164
Train RMSE is 0.24033934554311154
Train r2 is 0.8931871733593469

Test MSE is 0.1049176506666925
Test RMSE is 0.323909942216494
Test r2 is 0.7987191951921476

Fold number 5

Train MSE is 0.05877043774050492
Train RMSE is 0.242426149044415
Train r2 is 0.8894420344817248

Test MSE is 0.0999443859268098
Test RMSE is 0.3161

In [85]:
n = np.array([0.3253315255976139,0.3066586796388206,0.3134654739686835,0.31743078391525414, 0.3146086277055311 ])
n.var()

3.6697920750480056e-05

# Random Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV

start_time = time.time()

kf = KFold()
RMSE_train = []
RMSE_test = []
count = 0

time_start = time.time()

fold_num = kf.get_n_splits(data)

for train_index, test_index in kf.split(data):

    X_train, X_test = X.loc[train_index, predictors], X.loc[test_index, predictors]
    y_train, y_test = y.loc[train_index, response], y.loc[test_index, response]

    rf = RandomForestRegressor(n_estimators = 10000, min_samples_split = 2, min_samples_leaf = 1,
                                        max_features = 130, max_depth = 70)

    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    y_pred_train = rf.predict(X_train)
    
    a = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE_train.append(a)
    b = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSE_test.append(b)
    
    count += 1 
    
    print(f'\nFold number {count}')

    print(f'\nTrain MSE is {(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train RMSE is {np.sqrt(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train r2 is {r2_score(y_train, y_pred_train)}')

    print(f'\nTest MSE is {(mean_squared_error(y_test, y_pred))}')
    print(f'Test RMSE is {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'Test r2 is {r2_score(y_test, y_pred)}')
    
time_end = time.time()
    
print(f'\nThe average Train RMSE is {np.array(RMSE_train).mean()} ')
print(f'The average Test RMSE is {np.array(RMSE_test).mean()} ')
print(f'The variance Test RMSE is {np.array(RMSE_test).var()} ')
print(f'\ntime taken is {(time_end - time_start)/60} minutes')

C:\Users\susum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Fold number 1

Train MSE is 0.01613529828202777
Train RMSE is 0.127024793965697
Train r2 is 0.9696925558217718

Test MSE is 0.12767307786653903
Test RMSE is 0.35731369672395574
Test r2 is 0.7696752584917129


C:\Users\susum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Fold number 2

Train MSE is 0.01664696668780545
Train RMSE is 0.12902312462425272
Train r2 is 0.9693929001451855

Test MSE is 0.11425608050923343
Test RMSE is 0.3380178701033918
Test r2 is 0.7754224757426388


C:\Users\susum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Fold number 3

Train MSE is 0.016425668864494175
Train RMSE is 0.12816266564212128
Train r2 is 0.9693415866705516

Test MSE is 0.1186776305142483
Test RMSE is 0.3444961981129085
Test r2 is 0.7807907157468703


C:\Users\susum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Fold number 4

Train MSE is 0.0162919247528688
Train RMSE is 0.12763982432167792
Train r2 is 0.9698736820514923

Test MSE is 0.12157589863331832
Test RMSE is 0.34867735606620387
Test r2 is 0.7667609352034338


C:\Users\susum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Fold number 5

Train MSE is 0.016306534773427295
Train RMSE is 0.12769704293141362
Train r2 is 0.9693244192401068

Test MSE is 0.12226463556852452
Test RMSE is 0.349663603436967
Test r2 is 0.7809106754839512

The average Train RMSE is 0.12790949029703252 
The average Test RMSE is 0.34763374488868537 
The variance Test RMSE is 4.02440330411321e-05 

time taken is 2.7452242771784463 minutes


In [84]:
n = np.array([0.3579012047950553,0.3439826351600344,0.34979979463720834,0.3536739325995688, 0.3504700421725049 ])
n.var()

2.112086264546295e-05

# Stacking Model

In [19]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR        
from pygam import LinearGAM
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import StackingRegressor
from matplotlib import pyplot
from sklearn.svm import SVR

In [20]:
# The following code was improvised from:
# https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/
def get_stacking(meta_learner):
    
    #define the base models
    level0 = list()
    level0.append(('lgbm', lgb.LGBMRegressor(objective='regression', subsample = 0.8, num_leaves = 24, 
                                       n_estimators = 500, learning_rate = 0.05)))
    level0.append(('rf',  RandomForestRegressor(n_estimators = 1800, min_samples_split = 5, min_samples_leaf = 2,
                                        max_features = 'auto', max_depth = 70, bootstrap = True)))
    level0.append(('xgb', xgb.XGBRegressor(colsample_bytree = 0.306, subsample = 1.0, n_estimators = 1975, max_depth = 6,
                            learning_rate = 0.05, reg_lambda = 1000)))
    level0.append(('gbr',GradientBoostingRegressor(learning_rate= 0.05, max_depth = 4, n_estimators= 1500, subsample = 0.8)))
    
    # define meta learner model
    level1 = meta_learner

    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5, verbose=2, n_jobs=-1)
    return model

In [21]:
start_time = time.time()

kf = KFold()
RMSE_train = []
RMSE_test = []
count = 0

time_start = time.time()

fold_num = kf.get_n_splits(data)

for train_index, test_index in kf.split(data):

    X_train, X_test = X.loc[train_index, predictors], X.loc[test_index, predictors]
    y_train, y_test = y.loc[train_index, response], y.loc[test_index, response]

    stacking = get_stacking(None)

    stacking.fit(X_train, y_train)

    y_pred = stacking.predict(X_test)
    y_pred_train = stacking.predict(X_train)
    
    a = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE_train.append(a)
    b = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSE_test.append(b)
    
    count += 1 
    
    print(f'\nFold number {count}')

    print(f'\nTrain MSE is {(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train RMSE is {np.sqrt(mean_squared_error(y_train, y_pred_train))}')
    print(f'Train r2 is {r2_score(y_train, y_pred_train)}')

    print(f'\nTest MSE is {(mean_squared_error(y_test, y_pred))}')
    print(f'Test RMSE is {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'Test r2 is {r2_score(y_test, y_pred)}')
    
time_end = time.time()
    
print(f'\nThe average Train RMSE is {a.mean()} ')
print(f'\nThe average Test RMSE is {b.mean()} ')
print(f'\ntime taken is {(time_end - time_start)/60} minutes')

C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py:642: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 1

Train MSE is 0.041426129878821265
Train RMSE is 0.20353410003933312
Train r2 is 0.9221879820950777

Test MSE is 0.10352908888618163
Test RMSE is 0.3217593648772039
Test r2 is 0.8132314890910317


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py:642: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 2

Train MSE is 0.04226587669239549
Train RMSE is 0.2055866646754976
Train r2 is 0.922289992367014

Test MSE is 0.09226334577938719
Test RMSE is 0.30374882021069183
Test r2 is 0.8186505813739938


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py:642: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 3

Train MSE is 0.04456968150650462
Train RMSE is 0.21111532750253975
Train r2 is 0.9168109543141962

Test MSE is 0.0959266407940213
Test RMSE is 0.3097202621625219
Test r2 is 0.8228140368311458


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py:642: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 4

Train MSE is 0.04383036959502691
Train RMSE is 0.20935703856098775
Train r2 is 0.9189507887956655

Test MSE is 0.10101832792791202
Test RMSE is 0.3178338055146306
Test r2 is 0.8061999080567606


C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py:642: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Fold number 5

Train MSE is 0.040963571800654554
Train RMSE is 0.20239459429701812
Train r2 is 0.9229400131637806

Test MSE is 0.09652312131658834
Test RMSE is 0.3106817041870801
Test r2 is 0.8270375947133151

The average Train RMSE is 0.20239459429701812 

The average Test RMSE is 0.3106817041870801 

time taken is 41.526736652851106 minutes


In [22]:
print(f'\nThe average Train RMSE is {np.array(RMSE_train).mean()} ')
print(f'The average Test RMSE is {np.array(RMSE_test).mean()} ')
print(f'The variance Test RMSE is {np.array(RMSE_test).var()} ')
print(f'\ntime taken is {(time_end - time_start)/60} minutes')


The average Train RMSE is 0.20639754501507523 
The average Test RMSE is 0.31274879139042566 
The variance Test RMSE is 4.0298424646107346e-05 

time taken is 41.526736652851106 minutes
